In [2]:
!pip install geomle

In [1]:
import sys
import os
from os import path
current_folder = path.dirname(path.abspath('')) 
sys.path.append(current_folder)
from estimators import *
from geomle import geomle, mle, DataGenerator
import multiprocessing as mp
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from scipy.linalg import cholesky
from scipy.special import gammainc, lambertw
import scipy.io
import matplotlib as mpl
from matplotlib import pyplot as plt
import umap
import seaborn as sns
import random
import time
import numpy as np
import pandas as pd
import pickle
import rpy2
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages
from functools import wraps
import subprocess
from IPython.display import display_html
from operator import itemgetter
ig0 = itemgetter(0)
ig1 = itemgetter(1)
ig2 = itemgetter(2)
rpy2.robjects.numpy2ri.activate()
utils = rpackages.importr('utils')
#utils.install_packages('intrinsicDimension')
#utils.install_packages('ider')
intdimr = rpackages.importr('intrinsicDimension')
ider   = rpackages.importr('ider')
r_base = rpackages.importr('base')

In [2]:
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

def mean_sqe(estimations, truth):
    '''
    Mean squared error 
    '''
    return ((estimations - truth)^2/truth).sum() /len(truth) 
    
def mean_pe(estimations, truth):
    '''
    Mean percentage error 
    '''
    return (abs(estimations - truth)/truth).sum() /len(truth)*100

def mean_ge(estimations, truth):
    '''
    Mean geometric error: The geometric mean of the error *ratio*. It is always >= 1.
    '''
    ratios = np.concatenate(((estimations/truth)[np.newaxis, :], (truth/estimations)[np.newaxis, :]), axis=0)
    return np.power(ratios.max(axis=0).prod(), 1.0/len(estimations))

def med_pe(estimations, truth):
    '''
    Median error in %.
    '''
    return np.percentile(abs(estimations - truth)/truth, q=50)*100


def randball(n_points,ndim,radius,center = []):
    ''' Generate uniformly sampled ndim-sphere interior'''
    if center == []:
        center = np.array([0]*ndim)
    r = radius
    x = np.random.normal(size=(n_points, ndim))
    ssq = np.sum(x**2,axis=1)
    fr = r*gammainc(ndim/2,ssq/2)**(1/ndim)/np.sqrt(ssq)
    frtiled = np.tile(fr.reshape(n_points,1),(1,ndim))
    p = center + np.multiply(x,frtiled)
    return p, center

def proxy(tup):
    function,X,Dict = tup
    return function(X,**Dict)

def get_nn(X,k,n_jobs=1):
    neigh = NearestNeighbors(n_neighbors=k,n_jobs=n_jobs)
    neigh.fit(X)
    dists, inds = neigh.kneighbors(return_distance=True)
    return dists,inds

def asPointwise(data,function, params, precomputed_knn = None, n_neighbors=100, n_jobs=1):
    '''Use a global estimator as a pointwise one by creating kNN neighborhoods'''
    if precomputed_knn is not None:
        knn = precomputed_knn
    else:
        _, knn = get_nn(data, k=n_neighbors, n_jobs=n_jobs)
        
    if n_jobs > 1:
        pool = mp.Pool(n_jobs)
        results = pool.map(proxy,[(function,data[i,:],params) for i in knn])
        pool.close()
        return results
    else:
        return [function(data[i,:],**params) for i in knn]


from functools import wraps
def calculate_time(func): 
    @wraps(func)
    def inner_func(*args, **kwargs): 
        begin = time.time() 
        res = func(*args, **kwargs) 
        end = time.time()
        return res, end - begin
    return inner_func

class DimEst():
    def __init__(self):
        self.names = ['MLE', 'GeoMLE', 'MIND', 'DANCo', 'FastDANCo', 'ESS', 'PCA', 'CD','FisherS','ANOVA','TwoNN']
        self.caldatas = {}
        
    def estimateAllMethods(self, data,ConditionalNumber=10):
        dim = data.shape[1]
        self.funcs = {'MLE':          self.mle(data),
                      #'GeoMLE':       self.geomle(data, dim),
                      #'DANCo':        self.danco(data, dim),
                      'FastDANCo':    self.fast_danco(data),
                      #'ESS':          self.ess(data),
                      'PCA':          self.pca(data),
                      #'CD':           self.cd(data),
                      'FisherS':      self.fisherS(data,ConditionalNumber),
                      'ANOVA':        self.anova(data),
                      'TwoNN':        self.twonn(data)
                     }
                      
        self.times = {key: ig1(val) for key, val in self.funcs.items()}
        self.funcs = {key: ig0(val) for key, val in self.funcs.items()}
        return self.funcs, self.times
    
    def estimateAllMethodsLocally(self, data, k, n_jobs = 1, ConditionalNumber = 10):
        dim = data.shape[1]
        
        _, knn = get_nn(data, k, n_jobs)
        
        mle_pw, tle_pw, mom_pw, ed_pw, ged_pw, pca_pw = self.rado_ests(data,k).values()
        self.funcs = {'MLE':          self.mlelocal(data,k),
                      #'GeoMLE':       self.geomlelocal(data, dim),
                      'mind_mlk':         asPointwise(data,self.mind_mlk,{'dim':dim},precomputed_knn=knn,n_jobs=1),
                      'mind_mli':         asPointwise(data,self.mind_mli,{'dim':dim},precomputed_knn=knn,n_jobs=1),
                      #'DANCo':        asPointwise(data,self.danco,{'dim':dim},precomputed_knn=knn,n_jobs=1),
                      'FastDANCo':    self.fast_dancoloop(data),
                      'ESS':          asPointwise(data,self.ess,{},precomputed_knn=knn,n_jobs=n_jobs),
                      #'PCA':          self.pca(data),
                      'CD':           asPointwise(data,self.lcd,{},precomputed_knn=knn,n_jobs=n_jobs),
                      'FisherS':      asPointwise(data,self.fisherS,{'ConditionalNumber':ConditionalNumber},precomputed_knn=knn,n_jobs=n_jobs),
                      'ANOVA':        self.anovalocal(data,k),
                      'TwoNN':        asPointwise(data,self.twonn,{},precomputed_knn=knn,n_jobs=n_jobs),
                      'radoMLE':      mle_pw,
                      'radoTLE':      tle_pw,
                      'radoMOM':      mom_pw,
                      'radoED':       ed_pw,
                      'radoGED':      ged_pw,
                      'radoPCA':      pca_pw
                     }
                      
        self.times = {}
        for key, val in self.funcs.items():
            if key in ['MLE','ANOVA','FastDANCo']:
                self.funcs[key] = np.array(val[0])
                self.times[key] = val[1]
            elif 'rado' in key:
                self.funcs[key] = np.array(val)
            else:
                self.funcs[key] = np.array([i[0] for i in val])
                self.times[key] = np.sum([i[1] for i in val])
            
        return self.funcs, self.times
    
    
    @staticmethod
    def rado_ests(data,k):
        return radovanovic_estimators_matlab(data,k=k)
        mle_pw, tle_pw, mom_pw, ed_pw, ged_pw, pca_pw = rado_ests.values()
    
    @staticmethod
    @calculate_time
    def mle(data):
        return intdimr.maxLikGlobalDimEst(data,k=20).rx2('dim.est')[0]
    
    @staticmethod
    @calculate_time
    def mlelocal(data,k):
        res = intdimr.maxLikPointwiseDimEst(data,k=k)
        return np.array([i[0] for i in res])

    @staticmethod
    @calculate_time
    def geomle(data, dim):
#         k1 =  k1_log(dim)
#         k2 =  k2_log(dim)
        return geomle(pd.DataFrame(data), k1=20, k2=55, nb_iter1=1, alpha=5e-3).mean()

    @staticmethod
    @calculate_time
    def geomlelocal(data, dim):
#         k1 =  k1_log(dim)
#         k2 =  k2_log(dim)
        return geomle(pd.DataFrame(data), k1=20, k2=55, nb_iter1=1, alpha=5e-3)
    
    @staticmethod
    @calculate_time
    def mind_mlk(data, dim):
        return intdimr.dancoDimEst(data, k=10, D=min(dim,100), ver="MIND_MLk").rx2('dim.est')[0]
    

    @staticmethod
    @calculate_time
    def mind_mli(data, dim):
        return intdimr.dancoDimEst(data, k=10, D=min(dim,100), ver="MIND_MLi").rx2('dim.est')[0]
    
    #@staticmethod
    @calculate_time
    def danco(self,data, dim):
        try:
            res = intdimr.dancoDimEst(data, k=10, D=min(dim,100), calibration_data = self.caldatas[len(data)], ver="DANCo")
            self.caldatas[len(data)]=res[2]
            return res.rx2('dim.est')[0]
        except:
            res = intdimr.dancoDimEst(data, k=10, D=min(dim,100), ver="DANCo")
            self.caldatas[len(data)]=res[2]
            return res.rx2('dim.est')[0]

    @staticmethod
    @calculate_time
    def fast_danco(data):
        return runDANCo(data)[0]
    
    @staticmethod
    @calculate_time
    def fast_dancoloop(data):
        return runDANColoop(data)
    
    @staticmethod
    @calculate_time
    def ess(data):
        return ess_py(data)[0]
    
    @staticmethod
    @calculate_time
    def pca(data):
        return intdimr.pcaLocalDimEst(data, 'FO').rx2('dim.est')[0]
    
    @staticmethod
    @calculate_time
    def cd(data):
        return corint_py(data, k1=10, k2=20)[0]
    
    @staticmethod
    @calculate_time
    def lcd(data):
        return corint_py(data, k1=10, k2=len(data)-1)[0]
    
    @staticmethod
    @calculate_time
    def fisherS(data,ConditionalNumber):
        return SeparabilityAnalysis(data,ProducePlots=0,alphas=np.arange(.2,1,.02)[None],ConditionalNumber=ConditionalNumber)[1][0]
    
    @staticmethod
    @calculate_time
    def anova(data):
        return runANOVAglobal(data)[0,0]
    
    @staticmethod
    @calculate_time
    def anovalocal(data,k):
        return runANOVAlocal(data,k=k)[:,0]
    
    @staticmethod
    @calculate_time
    def twonn(data):
        res = twonn_py(data)
        return res    

# Synthetic data

In [3]:
DE=DimEst()

In [4]:
all_data_files = list(filter(lambda x: '.data' in x, os.listdir('../data/id-tle-synth-m10000-data/data/m10000/')))

unique_datasets = np.unique([i.split('-')[0]+'-' for i in all_data_files])
unique_datasets_00 = [file+'00.data' for file in unique_datasets]

synthetic_data = [np.array(pd.read_csv('../data/id-tle-synth-m10000-data/data/m10000/'+file,sep=' ',header=None)) for file in unique_datasets_00]
synthetic_data = dict(zip(unique_datasets_00,synthetic_data))

list(zip(list(synthetic_data.keys()),[i.shape for i in synthetic_data.values()]))

[('m1-00.data', (10000, 11)),
 ('m10a-00.data', (10000, 11)),
 ('m10b-00.data', (10000, 18)),
 ('m10c-00.data', (10000, 25)),
 ('m11-00.data', (10000, 3)),
 ('m12-00.data', (10000, 20)),
 ('m13-00.data', (10000, 13)),
 ('m2-00.data', (10000, 5)),
 ('m3-00.data', (10000, 6)),
 ('m4-00.data', (10000, 8)),
 ('m5-00.data', (10000, 3)),
 ('m6-00.data', (10000, 36)),
 ('m7-00.data', (10000, 3)),
 ('m8-00.data', (10000, 72)),
 ('m9-00.data', (10000, 20))]

### Global ID saturation analysis

In [ ]:
# testing separability saturation
n_repeats = 2
all_sample_sizes = [1,2,4,8,10,20,40,60,80]

for key,data in synthetic_data.items():
    
    datasets_done = [i.split('_')[0] for i in list(filter(lambda x: '.data' in x, os.listdir('../results')))]
    dataset_name = key
    n_samples = data.shape[0]
    
    if dataset_name in datasets_done:
        print('already computed ', dataset_name)
        continue
    
    print('\n',dataset_name)
    print('Running subsampling analysis...\nSubsample percentages = {}\nNumber of repeats = {}\nNumber of samples = {}\nDimension = {}'.format(all_sample_sizes,n_repeats,n_samples,data.shape[1]))
    print('----------------------------\n')
    seed = 0; np.random.seed(seed)
    print(f'Random seed set to {seed} before running the test')
    
    n_methods = 6
    all_dim_estimates = np.empty([n_methods,len(all_sample_sizes)+1,n_repeats])

    runtimes = []
    for i,sz in enumerate(all_sample_sizes):
        sample_size = int(n_samples*sz/100)
        print('Sample size = {}'.format(sample_size))
        start_time = time.time()
        for j in range(0,n_repeats):
            sample = np.random.choice(n_samples,replace=False, size=sample_size)
            xs = data[sample,:]

            #Run estimators
            allres = DE.estimateAllMethods(xs)
            results = allres[0]
            runtimes.append(allres[1])

            #Store
            for it,key in enumerate(results.keys()):
                all_dim_estimates[it,i,j] =  results[key]


        print("Elapsed time = {}".format(time.time()-start_time))

    allres = DE.estimateAllMethods(data)
    results = allres[0]
    runtimes.append(allres[1])

    for i in range(0,n_repeats):
        for it,key in enumerate(results.keys()):
            all_dim_estimates[it,len(all_sample_sizes),i] =  results[key]

    with open("../results/"+dataset_name+"_all_dim_estimates.npy",'wb') as f:
        pickle.dump(all_dim_estimates,f)
        
all_sample_sizes.append(100)
np.savetxt("../results/all_sample_sizes.txt", all_sample_sizes, delimiter="\t")
with open("../results/all_dim_estimates_keys.npy",'wb') as f:
        pickle.dump(list(results.keys()),f)

already computed  m1-00.data
already computed  m10a-00.data
already computed  m10b-00.data

 m10c-00.data
Running subsampling analysis...
Subsample percentages = [1, 2, 4, 8, 10, 20, 40, 60, 80]
Number of repeats = 2
Number of samples = 10000
Dimension = 25
----------------------------

Random seed set to 0 before running the test
Sample size = 100
Elapsed time = 14.658308982849121
Sample size = 200
Elapsed time = 13.042473077774048
Sample size = 400
Elapsed time = 13.147257089614868
Sample size = 800
Elapsed time = 13.323329210281372
Sample size = 1000
Elapsed time = 13.56218934059143
Sample size = 2000
Elapsed time = 15.96231722831726
Sample size = 4000
Elapsed time = 26.25061559677124
Sample size = 6000
Elapsed time = 33.37805151939392
Sample size = 8000
Elapsed time = 54.82598066329956

 m11-00.data
Running subsampling analysis...
Subsample percentages = [1, 2, 4, 8, 10, 20, 40, 60, 80]
Number of repeats = 2
Number of samples = 10000
Dimension = 3
----------------------------

Rand

In [ ]:
# plot convergence curve
alls=pd.read_csv('../results/all_sample_sizes.txt', sep='\t',header=None)
all_sample_sizes = alls.to_numpy()[:,0]

with open("../results/"+dataset_name+"_all_dim_estimates.npy",'rb') as f:
    all_dim_estimates = pickle.load(f)
    
with open("../results/all_dim_estimates_keys.npy",'rb') as f:
    estimators = pickle.load(f)

for i,estimator in enumerate(estimators):
    dim_estimates = all_dim_estimates[i]
    
    mn = np.mean(dim_estimates[:,:],1)
    std = np.std(dim_estimates[:,:],1)

    plt.figure(figsize=(10,3))
    plt.subplot(121)
    plt.title(estimator)
    plt.plot(all_sample_sizes,mn,'bs-')
    plt.plot(all_sample_sizes,mn-std,'r--')
    plt.plot(all_sample_sizes,mn+std,'r--')
    plt.plot(all_sample_sizes,dim_estimates,'b+')
    plt.xlabel('Percentage of points')
    plt.ylabel('Estimated intrinsic dimension')

### Local estimates convergence

In [ ]:
# testing separability saturation
n_jobs = 4
num_neighbors = num_neighbors = np.arange(50,450,50)
n_repeats = 2
all_sample_sizes = [1,2,4,8,10,20,40,60,80]


for n_neighbors in num_neighbors:

    for key,data in synthetic_data.items():

        datasets_done = [i.split('_')[0] for i in list(filter(lambda x: '.data' in x, os.listdir('../results')))]
        dataset_name = key
        n_samples = data.shape[0]

        if dataset_name in datasets_done:
            print('already computed ', dataset_name)
            continue

        print('\n',dataset_name)
        print('Running subsampling analysis...\nSubsample percentages = {}\nNumber of repeats = {}\nNumber of samples = {}\nDimension = {}'.format(all_sample_sizes,n_repeats,n_samples,data.shape[1]))
        print('----------------------------\n')
        print('kNN = ',n_neighbors)
        seed = 0; np.random.seed(seed)
        print(f'Random seed set to {seed} before running the test')


        n_methods = 16
        all_dim_estimates = np.empty([n_methods,len(all_sample_sizes)+1,n_repeats,len(data)])

        runtimes = []
        for i,sz in enumerate(all_sample_sizes):
            sample_size = int(n_samples*sz/100)
            print('Sample size = {}'.format(sample_size))
            start_time = time.time()
            for j in range(0,n_repeats):
                sample = np.random.choice(n_samples,replace=False, size=sample_size)
                xs = data[sample,:]

                #Run estimators
                allres = DE.estimateAllMethodsLocally(xs, k = n_neighbors, n_jobs = n_jobs, ConditionalNumber = 10)

                results = allres[0]
                runtimes.append(allres[1])

                #Store
                for it,key in enumerate(results.keys()):
                    all_dim_estimates[it,i,j,:len(xs)] =  results[key]


            print("Elapsed time = {}".format(time.time()-start_time))

        allres = DE.estimateAllMethodsLocally(data, k = n_neighbors, n_jobs = n_jobs, ConditionalNumber = 10)

        results = allres[0]
        runtimes.append(allres[1])

        for i in range(0,n_repeats):
            for it,key in enumerate(results.keys()):
                all_dim_estimates[it,len(all_sample_sizes),i,:len(data)] =  results[key]

        with open("../results/"+dataset_name+"_all_dim_estimates_kNN"+str(n_neighbors)+".npy",'wb') as f:
            pickle.dump(all_dim_estimates,f)

with open("../results/"+dataset_name+"_all_dim_estimates_keys_kNN"++str(n_neighbors)+".npy",'wb') as f:
        pickle.dump(list(results.keys()),f)

In [ ]:
# plot convergence curve (to modify for local id)
alls=pd.read_csv('../results/all_sample_sizes.txt', sep='\t',header=None)
all_sample_sizes = alls.to_numpy()[:,0]

with open("../results/"+dataset_name+"_all_dim_estimates.npy",'rb') as f:
    all_dim_estimates = pickle.load(f)
with open("../results/"+dataset_name+"_all_dim_estimates_keys_kNN"++str(n_neighbors)+".npy",'wb') as f:
    estimators = pickle.load(f)

for i,estimator in enumerate(estimators):
    dim_estimates = all_dim_estimates[i]
    
    mn = np.mean(dim_estimates[:,:],1)
    std = np.std(dim_estimates[:,:],1)

    plt.figure(figsize=(10,3))
    plt.subplot(121)
    plt.title(estimator)
    plt.plot(all_sample_sizes,mn,'bs-')
    plt.plot(all_sample_sizes,mn-std,'r--')
    plt.plot(all_sample_sizes,mn+std,'r--')
    plt.plot(all_sample_sizes,dim_estimates,'b+')
    plt.xlabel('Percentage of points')
    plt.ylabel('Estimated intrinsic dimension')